In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [25]:
import pandas as pd
import base64
import numpy as np
# import imageio
import os
import scipy
import gensim
import re
import nltk
from nltk.stem import WordNetLemmatizer
from nltk import word_tokenize
import gensim.corpora as corpora
import itertools
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
from nltk.corpus import stopwords
stops = stopwords.words('english')

''' This method removes all kinds of line breaks. '''
def removeLineBreaks(tweet):
    return re.sub("\n\r|\r\n|\n|\r"," ", tweet)

''' This method removes all the url's in the tweet'''
def removeURLs(tweet):
    return re.sub("(http|ftp|https)://([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:/~+#-]*[\w@?^=%&/~+#-])?", " ", tweet)

''' This method removes all emojis from the tweet'''
def removeEmojis(tweet):
    tweet = tweet.encode('ascii', 'ignore').decode('ascii')
    return tweet

''' This method checks if the tweet is a retweet or not.
    a retweet contains RT @***** '''
def isRetweet(tweet):
    retweet = re.compile("RT @[A-Za-z0-9]*:")
    retweet.match(tweet)

    return bool(re.search("RT @[A-Za-z0-9]*:", tweet))

''' This method removes the retweet tag from tweets'''
def removeRTtag(tweet):
    return re.sub("RT @[A-Za-z0-9]*: ", " ", tweet)

''' This method removes all the mentions.
    mentions are usually with @'''
def removeMentions(tweet):
    return re.sub("@[A-Za-z0-9]*", " ", tweet)

''' This method removes multiple spaces.'''
def removeMultipleSpaces(tweet):
    return re.sub(" +", " ", tweet)

''' This method turns the tweets into lowercase. '''
def lowercasetweet(tweet):
    return tweet.lower()

''' This method removes all the punctuations from the tweet.'''
def removePunctuations(tweet):
    return re.sub("[.,!'\";:?…]+", " ", tweet)

''' This method removes special characters from tweets.'''
def removeSpecialCharacters(tweet):
    return re.sub("[@#$%^*(){}\\\<>\[\]~/|=\+\-&_¿ߒ]+"," ", tweet)

''' This method removes alpha-numeric charcters from the tweet.'''
def removeAlphaNumeric(tweet):
    # return re.sub("[A-Za-z]+[0-9]+", "", tweet)
    return re.sub("[0-9]+", "", tweet)

''' Lemmatization using nltk. '''
def lemmatizeTweet(tweet):
    return [WordNetLemmatizer().lemmatize(token) for token in word_tokenize(tweet)]

def cleanData(text, lowercase = False, remove_stops = False, stemming = False, lemmatization = False):
    txt = str(text)

    # Replace apostrophes with standard lexicons
    txt = txt.replace("isn't", "is not")
    txt = txt.replace("aren't", "are not")
    txt = txt.replace("ain't", "am not")
    txt = txt.replace("won't", "will not")
    txt = txt.replace("didn't", "did not")
    txt = txt.replace("shan't", "shall not")
    txt = txt.replace("haven't", "have not")
    txt = txt.replace("hadn't", "had not")
    txt = txt.replace("hasn't", "has not")
    txt = txt.replace("don't", "do not")
    txt = txt.replace("wasn't", "was not")
    txt = txt.replace("weren't", "were not")
    txt = txt.replace("doesn't", "does not")
    txt = txt.replace("'s", " is")
    txt = txt.replace("'re", " are")
    txt = txt.replace("'m", " am")
    txt = txt.replace("'d", " would")
    txt = txt.replace("'ll", " will")

    # Emoji replacement
    txt = re.sub(r':\)',r' happy ',txt)
    txt = re.sub(r':D',r' happy ',txt)
    txt = re.sub(r':P',r' happy ',txt)
    txt = re.sub(r':\(',r' sad ',txt)

    # Replace words like sooooooo with so
    txt = ''.join(''.join(s)[:2] for _, s in itertools.groupby(txt))
    return txt


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [29]:
final = np.load('/content/gdrive/My Drive/IR Assignment/IR_assignment.npy',allow_pickle=True)
uniq_words=[]
count={}
import os
import pickle
index={}
cnt=1
for line in final:
    print(cnt)
    cnt+=1
    ll = lemmatizeTweet(removeMultipleSpaces(removeURLs(removeMentions(removeEmojis(removeSpecialCharacters(removePunctuations(removeAlphaNumeric(cleanData(removeLineBreaks(line[1].lower()))))))))))
    for word in ll:
        if word not in stops:
          if word not in uniq_words:
            uniq_words.append(word)
            count[word]=1
            index[word]=[]
          else:
            count[word]+=1
          if word not in index[word]:
             index[word].append(int(line[0]))


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [30]:
print(uniq_words)
print(count['sorry'])

['sorry', 'hear', 'state', 'thank', 'opening', 'personal', 'thought', 'college', 'perception', 'well', 'life', 'happy', 'party', 'atmosphere', 'visible', 'part', 'campus', 'guarantee', 'people', 'around', 'feel', 'way', 'try', 'connect', 'somehow', 'alone', 'get', 'better', 'done', 'however', 'enjoy', 'courseload', 'heavy', 'even', 'consider', 'taking', 'summer', 'course', 'might', 'finish', 'fast', 'would', 'like', 'still', 'faster', 'normal', 'take', 'elective', 'interested', 'relationship', 'know', 'easy', 'usually', 'tell', 'good', 'number', 'wish', 'single', 'matter', 'situation', 'somebody', 'jealous', 'love', 'loving', 'something', 'worth', 'pursuing', 'went', 'bad', 'parent', 'remember', 'learn', 'mistake', 'thing', 'time', 'important', 'talking', 'feeling', 'suicidal', 'afraid', 'contact', 'helplines', 'listed', 'subreddit', 'help', 'listen', 'want', 'going', 'wasting', 'calling', 'point', 'experience', 'see', 'privileged', 'enough', 'witness', 'world', 'doe', 'suicide', 'atte

In [0]:
pickle_out = open("/content/gdrive/My Drive/IR Assignment/inverted_index.pickle","wb")
pickle.dump(index, pickle_out)
pickle_out.close()
pickle_out = open("/content/gdrive/My Drive/IR Assignment/count_word.pickle","wb")
pickle.dump(index, pickle_out)
pickle_out.close()
np.save("/content/gdrive/My Drive/IR Assignment/uniq_words.npy",uniq_words)